In [1]:
sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.2 LTS

Matrix products: default
BLAS: /usr/lib/libblas/libblas.so.3.6.0
LAPACK: /usr/lib/lapack/liblapack.so.3.6.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.4.3      R6_2.2.2            magrittr_1.5       
 [4] IRdisplay_0.4.4     pbdZMQ_0.2-6        tools_3.4.3        
 [7] crayon_1.3.4        uuid_0.1-2          stringi_1.1.6      
[10] IRkernel_0.8.6.9000 jsonlite_1.5        stringr_1.2.0      
[13] digest_0.6.12     

In [2]:
library("DESeq2")
library("geneplotter")
library("ggplot2")
library("vsn")
library("genefilter")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min


Attaching packag

In [3]:
setwd("/home/gstupp/projects/Wolan/ana_extractions/analysis/analysis_deseq")

In [4]:
file_path = "X.csv"
countData = read.csv(file_path, row.names = 1)

In [5]:
meta_path = "metadata.csv"
sampleTable = read.csv(meta_path, row.names = 1)
sampleTable = sampleTable[colnames(countData),]
sampleTable$cell = factor(sampleTable$cell)
sampleTable$method = factor(sampleTable$method)
sampleTable$method <- relevel(sampleTable$method, ref="sonic")
sampleTable = sampleTable[!(rownames(sampleTable) %in% c("PA_pellet_1", "PA_pellet_2", "PA_pellet_3")),]
sampleTable

,cell,file,method,name.1,pretty_name
BS_sonic_1,BS,data/BS/Sonic/BS sonic 1.txt,sonic,BS_sonic_1,BS sonic 1
BS_sonic_2,BS,data/BS/Sonic/BS sonic 2.txt,sonic,BS_sonic_2,BS sonic 2
BS_sonic_3,BS,data/BS/Sonic/BS sonic 3.txt,sonic,BS_sonic_3,BS sonic 3
BS_triflic_1,BS,data/BS/Triflic/BS triflic 1.txt,triflic,BS_triflic_1,BS triflic 1
BS_triflic_2,BS,data/BS/Triflic/BS triflic 2.txt,triflic,BS_triflic_2,BS triflic 2
BS_triflic_3,BS,data/BS/Triflic/BS triflic 3.txt,triflic,BS_triflic_3,BS triflic 3
Jurkat_sonic_1,Jurkat,data/Jurkat/Sonic/Jurkat sonic 1.txt,sonic,Jurkat_sonic_1,Jurkat sonic 1
Jurkat_sonic_2,Jurkat,data/Jurkat/Sonic/Jurkat sonic 2.txt,sonic,Jurkat_sonic_2,Jurkat sonic 2
Jurkat_sonic_3,Jurkat,data/Jurkat/Sonic/Jurkat sonic 3.txt,sonic,Jurkat_sonic_3,Jurkat sonic 3
Jurkat_triflic_1,Jurkat,data/Jurkat/Triflic/Jurkat triflic 1.txt,triflic,Jurkat_triflic_1,Jurkat triflic 1


## BS

In [6]:
sampleTableX = sampleTable[sampleTable$cell=="BS",]
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 2, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,BS_sonic_1,BS_sonic_2,BS_sonic_3,BS_triflic_1,BS_triflic_2,BS_triflic_3
C0H3V2,4,7,5,0,3,4
C0H3V3,6,6,9,0,0,0


In [7]:
dds = DESeq(dds)
res = results(dds)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 1372 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 264, 19% 
LFC < 0 (down)   : 228, 17% 
outliers [1]     : 0, 0% 
low counts [2]   : 266, 19% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [8]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="BS_deseq_results.csv")
resOrdered[0:20,]

log2 fold change (MLE): method triflic vs sonic 
Wald test p-value: method triflic vs sonic 
DataFrame with 20 rows and 6 columns
        baseMean log2FoldChange     lfcSE      stat       pvalue         padj
       <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
P80239  747.5681      -4.366607 0.2995649 -14.57650 3.963404e-48 4.383525e-45
P54375  329.4090       5.712140 0.4374335  13.05830 5.698257e-39 3.151136e-36
P0CI73  174.4533       5.838238 0.4573601  12.76508 2.568842e-37 9.470464e-35
P26901  280.4514      -3.304884 0.2700305 -12.23893 1.925572e-34 5.324205e-32
P94391  180.9387      -4.243002 0.3568728 -11.88940 1.343752e-32 2.972380e-30
...          ...            ...       ...       ...          ...          ...
P94550  47.60762      -4.012102 0.5237998 -7.659609 1.865000e-14 1.289181e-12
P54542  59.62276      -3.447706 0.4536070 -7.600644 2.946601e-14 1.917024e-12
P45745  61.68154       9.307479 1.2447275  7.477523 7.573639e-14 4.408655e-12
P54716  48.0

In [9]:
sizeFactors(dds)

BS_sonic_1   BS_sonic_2   BS_sonic_3 BS_triflic_1 BS_triflic_2 BS_triflic_3 
   0.9845253    0.9086815    0.9433868    0.8298068    1.0597155    1.3676984

In [10]:
write.csv(counts(dds, normalized=TRUE), "X_bs_normalized.csv")

## Jurkat

In [16]:
sampleTableX = sampleTable[sampleTable$cell=="Jurkat",]
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 2, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,Jurkat_sonic_1,Jurkat_sonic_2,Jurkat_sonic_3,Jurkat_triflic_1,Jurkat_triflic_2,Jurkat_triflic_3
A0A024QZP7,24,3,6,12,10,27
A0A024R216,0,0,6,0,0,6


In [17]:
dds = DESeq(dds)
res = results(dds)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 5272 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 23, 0.44% 
LFC < 0 (down)   : 48, 0.91% 
outliers [1]     : 3, 0.057% 
low counts [2]   : 3066, 58% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [18]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="Jurkat_deseq_results.csv")
resOrdered[0:20,]

log2 fold change (MLE): method triflic vs sonic 
Wald test p-value: method triflic vs sonic 
DataFrame with 20 rows and 6 columns
        baseMean log2FoldChange     lfcSE      stat       pvalue         padj
       <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
H3BR70 226.09501      -3.799501 0.6992692 -5.433531 5.524981e-08 0.0001217153
B4DNK4 286.15842      -3.439163 0.6913746 -4.974385 6.545502e-07 0.0007209870
Q9UII2  24.40724      -7.667422 1.5904925 -4.820785 1.429945e-06 0.0010500564
M0QY96  24.48991       5.146161 1.1247024  4.575576 4.749116e-06 0.0026155757
M0R2I7  25.25797       4.262155 0.9641867  4.420467 9.848793e-06 0.0043393780
...          ...            ...       ...       ...          ...          ...
P09104 111.74183      -2.889324 0.7956442 -3.631428 0.0002818575   0.03880826
M0QYM7  69.18233      -3.504917 0.9869576 -3.551233 0.0003834308   0.04968812
E9PK25  77.41391      -3.100875 0.8966799 -3.458174 0.0005438493   0.05457674
E9PQB7  73.0

In [19]:
sizeFactors(dds)

Jurkat_sonic_1   Jurkat_sonic_2   Jurkat_sonic_3 Jurkat_triflic_1 
       1.6064868        1.0508626        1.4520681        0.7459527 
Jurkat_triflic_2 Jurkat_triflic_3 
       0.6991257        0.8583489

In [20]:
head(counts(dds, normalized=TRUE))

,Jurkat_sonic_1,Jurkat_sonic_2,Jurkat_sonic_3,Jurkat_triflic_1,Jurkat_triflic_2,Jurkat_triflic_3
A0A024QZP7,14.939432,2.854798,4.132038,16.086811,14.303579,31.455741
A0A024R216,0.000000,0.000000,4.132038,0.000000,0.000000,6.990165
A0A024R341,0.000000,0.000000,4.132038,0.000000,0.000000,0.000000
A0A024R4M0,3.734858,1.903198,8.952748,26.811352,5.721431,3.495082
A0A024R571,2.489905,5.709595,4.132038,8.043406,7.151789,2.330055
A0A024RA52,4.979811,12.370790,3.443365,9.383973,5.721431,3.495082


In [21]:
head(counts(dds))

,Jurkat_sonic_1,Jurkat_sonic_2,Jurkat_sonic_3,Jurkat_triflic_1,Jurkat_triflic_2,Jurkat_triflic_3
A0A024QZP7,24,3,6,12,10,27
A0A024R216,0,0,6,0,0,6
A0A024R341,0,0,6,0,0,0
A0A024R4M0,6,2,13,20,4,3
A0A024R571,4,6,6,6,5,2
A0A024RA52,8,13,5,7,4,3


In [22]:
write.csv(counts(dds, normalized=TRUE), "X_jurkat_normalized.csv")